<a href="https://colab.research.google.com/github/alialtinok/AI/blob/main/ANN_keras_Train_Test_Final_FeatureSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Google Drive Baglantisi**


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/Colab Notebooks/Artificial Neural Network

/content/gdrive/My Drive/Colab Notebooks/Artificial Neural Network


**Verileri Oluştur**

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import MinMaxScaler        
import numpy

numpy.random.seed(7)
dataset=numpy.loadtxt("Final_Veriseti.csv", delimiter=",")

datalar=dataset[:,0:16]
datalar=MinMaxScaler().fit_transform(datalar)

etiketler=dataset[:,16]
etiketler=np_utils.to_categorical(etiketler, 7)

# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(datalar, etiketler, test_size=0.2, random_state=0)


In [ ]:
print(etiketler.shape)
print(etiketler[13600,:])
print(datalar.shape)
#print(datalar)

(13611, 7)
[0. 0. 0. 0. 0. 0. 1.]
(13611, 16)


In [ ]:
#Sequential Feature Selection
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier

sfs = SequentialFeatureSelector(estimator=KNeighborsClassifier(n_neighbors=3),
                    k_features=12, 
                    forward=True, 
                    floating=False,
                    scoring='accuracy') 
X_new=sfs.fit_transform(X_train, Y_train)
print(X_train.shape)
print(X_new.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


(10888, 16)
(10888, 12)


In [ ]:
print(sfs.subsets_[5])


{'feature_idx': (1, 10, 12, 13, 15), 'cv_scores': array([0.91414141, 0.90312213, 0.91781451, 0.90996785, 0.91134589]), 'avg_score': 0.911278357551377, 'feature_names': ('1', '10', '12', '13', '15')}


In [ ]:
#PCA: Principle Component Analysis
from sklearn.decomposition import PCA

pca = PCA(n_components=10) 
fit = pca.fit(X_train)
X_train_new = fit.transform(X_train)
X_test_new =fit.transform(X_test)
# summarize components
#print("Explained Variance: %s" % fit.explained_variance_ratio_)
#print(fit.components_)

print(X_train.shape)
print(X_train_new.shape)

(10888, 16)
(10888, 10)


**MLP Yapisi Kurulumu**

In [ ]:
model=Sequential()
model.add(Dense(100, input_dim=16, activation='linear')) # Hidden Layer 1 (25 noron)
model.add(Dense(140, activation='relu'))              # Hidden Layer 2  (100 noron)
model.add(Dense(190, activation='swish'))              # Hidden Layer 2  (100 noron)

model.add(Dense(7, activation='sigmoid'))            #Output Layer

#Modeli Derleme
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=400, batch_size=100, verbose=1)
basarim=model.evaluate(X_train, Y_train)

print("\n Train %s : %.2f%% " % (model.metrics_names[1], basarim[1]*100))


Epoch 1/400
109/109 [==============================] - 1s 3ms/step - loss: 0.3760 - accuracy: 0.3944
Epoch 2/400
109/109 [==============================] - 0s 3ms/step - loss: 0.1092 - accuracy: 0.8786
Epoch 3/400
109/109 [==============================] - 0s 3ms/step - loss: 0.0817 - accuracy: 0.8939
Epoch 4/400
109/109 [==============================] - 0s 3ms/step - loss: 0.0743 - accuracy: 0.8985
Epoch 5/400
109/109 [==============================] - 0s 3ms/step - loss: 0.0694 - accuracy: 0.9072
Epoch 6/400
109/109 [==============================] - 0s 3ms/step - loss: 0.0665 - accuracy: 0.9087
Epoch 7/400
109/109 [==============================] - 0s 3ms/step - loss: 0.0624 - accuracy: 0.9170
Epoch 8/400
109/109 [==============================] - 0s 3ms/step - loss: 0.0618 - accuracy: 0.9181
Epoch 9/400
109/109 [==============================] - 0s 3ms/step - loss: 0.0605 - accuracy: 0.9165
Epoch 10/400
109/109 [==============================] - 0s 3ms/step - loss: 0.0640 - accura

In [ ]:
model.summary()
#model.get_config()
#from keras.utils.vis_utils import plot_model
#plot_model(model, to_file='model.png')

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 100)               1700      
_________________________________________________________________
dense_13 (Dense)             (None, 140)               14140     
_________________________________________________________________
dense_14 (Dense)             (None, 190)               26790     
_________________________________________________________________
dense_15 (Dense)             (None, 7)                 1337      
Total params: 43,967
Trainable params: 43,967
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.save("96.07.h5")

**Confusion Matrix**

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

_, test_acc = model.evaluate(X_test, Y_test, verbose=0)
print('>Test Accuracy: %.2f%%' % (test_acc * 100.0))


#yPredicted = model.predict(X_test)           
#yPredicted = model.predict_proba(X_test)     
yPredicted = model.predict_classes(X_test)
Y_Test_actual=numpy.argmax(Y_test,axis=1)

print(classification_report(y_true=Y_Test_actual, y_pred=yPredicted))
print(confusion_matrix(y_true=Y_Test_actual, y_pred=yPredicted))


>Test Accuracy: 92.66%
              precision    recall  f1-score   support

           0       0.95      0.96      0.95       376
           1       0.95      0.87      0.91       255
           2       1.00      1.00      1.00        92
           3       0.93      0.95      0.94       350
           4       0.97      0.95      0.96       390
           5       0.87      0.88      0.87       525
           6       0.92      0.93      0.92       735

    accuracy                           0.93      2723
   macro avg       0.94      0.93      0.94      2723
weighted avg       0.93      0.93      0.93      2723

[[360   1   0   1   0  12   2]
 [  5 222   0  19   3   6   0]
 [  0   0  92   0   0   0   0]
 [  1   9   0 334   3   3   0]
 [  0   1   0   7 370  10   2]
 [  3   1   0   0   4 462  55]
 [ 11   0   0   0   1  40 683]]


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
